In [ ]:
import pandas as pd
import warnings
import logging
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score, StratifiedGroupKFold
import random
import os
import numpy as np
from datetime import datetime


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64',
                'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


# Suppress warnings
warnings.filterwarnings('ignore')

# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

base_models = [
    ('xgb_2000', XGBClassifier(n_estimators=2000, max_depth=12, learning_rate=0.02, subsample=0.8, colsample_bytree=0.4, missing=-1, eval_metric='auc', tree_method='hist', device='cuda',)),
    ('xgb_2500', XGBClassifier(n_estimators=2500, max_depth=12, learning_rate=0.02, subsample=0.8, colsample_bytree=0.4, missing=-1, eval_metric='auc', tree_method='hist', device='cuda',)),
]

meta_model = XGBClassifier( 
    n_estimators=2500,
    max_depth=12, 
    learning_rate=0.02, 
    subsample=0.8,
    colsample_bytree=0.4, 
    missing=-1, 
    eval_metric='auc',
    tree_method='hist', 
    device='cuda',
)


def prepare_data(data, use_msisdn):
    logger.info('Preparing data...')
    if use_msisdn:
        X = data.drop(columns=['is_sa'])
    else:
        X = data.drop(columns=['is_sa', 'msisdn'])
    y = data['is_sa']
    groups = data['msisdn']
    logger.info('Data preparation completed.')
    return X, y, groups


def build_stacking_classifier(weight_adjustment=False):
    logger.info('Building stacking classifier...')
    final_estimator = meta_model
    if weight_adjustment:
        final_estimator.set_params(scale_pos_weight=(y == 0).sum() / (y == 1).sum())
    stacking_clf = StackingClassifier(
        estimators=base_models,
        final_estimator=final_estimator,
        n_jobs=-1,
    )
    logger.info('Stacking classifier built.')
    return stacking_clf


def build_voting_classifier():
    logger.info('Building voting classifier...')
    voting_clf = VotingClassifier(
        estimators=base_models,
        voting='soft',
        n_jobs=-1,
    )
    logger.info('Voting classifier built.')
    return voting_clf


def build_single_classifier(weight_adjustment=False):
    logger.info('Building single classifier...')
    model = XGBClassifier(
        n_estimators=2000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        tree_method='hist', 
        device='cuda',
        random_state=42,
    )
    if weight_adjustment:
        model.set_params(scale_pos_weight=(y == 0).sum() / (y == 1).sum())
    logger.info(f'Using model: {model}')
    return model


def build_model(mode='stacking', weight_adjustment=False):
    logger.info(f'Building model with mode: {mode}')
    if mode == 'stacking':
        return build_stacking_classifier(weight_adjustment)
    elif mode == 'single':
        return build_single_classifier(weight_adjustment)
    elif mode == 'voting':
        return build_voting_classifier()
    else:
        raise ValueError(f'Unknown mode: {mode}')


def cross_validate_model(model, X, y, groups, cv_strategy):
    logger.info('Starting cross-validation...')
    cv_scores = cross_val_score(
        model, X, y, cv=cv_strategy, groups=groups, scoring='f1')
    logger.info(f'Cross-validation completed. F1 score: {cv_scores.mean()}')
    return cv_scores


def predict(model, validation_data, output_path, use_msisdn=False):
    logger.info('Predicting and saving results...')
    msisdn = validation_data['msisdn']
    if not use_msisdn:
        validation_data = validation_data.drop(columns=['msisdn'])
    y_pred = model.predict(validation_data)
    result = pd.DataFrame({'msisdn': msisdn, 'is_sa': y_pred})
    result.to_csv(output_path, index=False)
    logger.info(f'Results saved to {output_path}')


def pseudo_labeling(train_data, validation_data, threshold=0.9, mode='voting', use_msisdn=False, weight_adjustment=False):
    logger.info('Starting pseudo-labeling...')
    X_train, y_train, groups_train = prepare_data(train_data, use_msisdn)
    X_unlabeled = validation_data.drop(columns=['msisdn']).copy(
    ) if not use_msisdn else validation_data.copy()

    model = build_model(mode, weight_adjustment)
    model.fit(X_train, y_train)

    y_unlabeled_pred = model.predict_proba(X_unlabeled)
    high_confidence_indices = (y_unlabeled_pred.max(axis=1) > threshold)

    pseudo_labeled_data = validation_data[high_confidence_indices].copy()
    pseudo_labeled_data['is_sa'] = y_unlabeled_pred[high_confidence_indices].argmax(
        axis=1)

    augmented_train_data = pd.concat(
        [train_data, pseudo_labeled_data], ignore_index=True)
    logger.info(
        f'Added {pseudo_labeled_data.shape[0]} pseudo-labeled samples.')
    logger.info('Pseudo-labeling completed.')
    return augmented_train_data


def adversarial_validation(train_data, validation_data, use_msisdn=False):
    logger.info('Starting adversarial validation...')
    train_data['is_train'] = 1
    validation_data['is_train'] = 0

    combined_data = pd.concat([train_data, validation_data], axis=0)
    if use_msisdn:
        X = combined_data.drop(columns=['is_sa', 'is_train'])
    else:
        X = combined_data.drop(columns=['is_sa', 'is_train', 'msisdn'])
    y = combined_data['is_train']

    adv_model = XGBClassifier(
        objective='binary:logistic', n_estimators=100, n_jobs=-1, random_state=42)
    adv_model.fit(X, y)

    feature_importances = adv_model.feature_importances_
    feature_names = X.columns
    important_features = [feature for feature, importance in zip(
        feature_names, feature_importances) if importance > np.median(feature_importances)]
    logger.info(
        f'Removed features: {set(feature_names) - set(important_features)}')
    logger.info('Adversarial validation completed.')
    if use_msisdn:
        train_data = train_data[important_features + ['is_sa'] + ['msisdn']]
        validation_data = validation_data[important_features + ['msisdn']]
    else:
        train_data = train_data[important_features + ['is_sa']]
        validation_data = validation_data[important_features]
    logger.info(
        f'Training on {train_data.shape[0]} samples with {train_data.shape[1] - (2 if use_msisdn else 1)} features')
    return train_data, validation_data


def main(dataset_version=40, use_pseudo_labeling=False, threshold=0.95, mode='voting', n_fold=5, need_prediction=False, use_adversarial_validation=False, use_msisdn=False, weight_adjustment=False):
    logger.info('Main function started.')
    seed_everything()
    output_path = f'/home/hwxu/Projects/Competition/Telecom/Output/submissions/pred-{dataset_version}_{datetime.now()}.csv'
    train_data = pd.read_csv(
        f'/home/hwxu/Projects/Competition/Telecom/Input/processed/train{dataset_version}.csv')
    validation_data = pd.read_csv(
        f'/home/hwxu/Projects/Competition/Telecom/Input/processed/val{dataset_version}.csv')
    train_data = reduce_mem_usage(train_data)
    validation_data = reduce_mem_usage(validation_data)
    logger.info(
        f"Training on {train_data.shape[0]} samples with {validation_data.shape[1]} features.")

    if use_adversarial_validation:
        train_data, validation_data = adversarial_validation(
            train_data, validation_data, use_msisdn)
    else:
        train_data = train_data
        validation_data = validation_data

    if use_pseudo_labeling:
        train_data = pseudo_labeling(
            train_data, validation_data, threshold, mode, use_msisdn, weight_adjustment)
    else:
        train_data = train_data

    X, y, groups = prepare_data(train_data, use_msisdn)
    cv_strategy = StratifiedGroupKFold(n_splits=n_fold)

    classifier = build_model(mode, weight_adjustment)

    cross_validate_model(classifier, X, y, groups, cv_strategy)

    if need_prediction:
        classifier.fit(X, y)
        predict(classifier, validation_data, output_path, use_msisdn)
    logger.info('Main function completed.')


In [ ]:
main(
    dataset_version=185, 
    use_pseudo_labeling=False, 
    threshold=0.95, 
    mode='voting', 
    n_fold=10, 
    need_prediction=True, 
    use_adversarial_validation=False, 
    use_msisdn=True, 
    weight_adjustment=False,
)
